In [24]:
# Cell 1 — Imports & connect
import sqlite3
import pandas as pd

# open (or create) your SQLite database
conn = sqlite3.connect('economic_data.db')


In [25]:
# Cell 2 — Read each CSV and push to SQL as raw tables
datasets = [
    ('gdp',        '../data_econ/gdp/gdp.csv'),
    ('emp',        '../data_econ/emp/emp.csv'),
    ('gdp_growth', '../data_econ/gdp_growth/gdp_growth.csv'),
]

for level, path in datasets:
    print(f"→ Loading raw table: {level}_raw from {path!r}")
    # adjust skiprows if your file header has fewer lines
    df = pd.read_csv(path, skiprows=4)
    
    # write the full wide-form DF into SQL
    df.to_sql(f"{level}_raw", conn, if_exists='replace', index=False)

print("All raw tables loaded into SQLite.")


→ Loading raw table: gdp_raw from '../data_econ/gdp/gdp.csv'
→ Loading raw table: emp_raw from '../data_econ/emp/emp.csv'
→ Loading raw table: gdp_growth_raw from '../data_econ/gdp_growth/gdp_growth.csv'
All raw tables loaded into SQLite.


In [29]:
import sqlite3

# 1) Connect & get a cursor
conn = sqlite3.connect('economic_data.db')
cur  = conn.cursor()

# 2) Define your filters (you may generate these programmatically, but we won't loop over tables)
target_countries = ['China','Japan','South Korea','United States','United Kingdom','Canada']
countries_sql = ", ".join(f"'{c}'" for c in target_countries)

years    = [str(y) for y in range(1990, 2024)]
columns  = ['Country Name', 'Indicator Name'] + years
col_clause = ", ".join(f'"{c}"' for c in columns)

# 3) Filter gdp_raw → gdp_filtered, drop old, rename
cur.execute(f"""
    CREATE TABLE gdp_filtered AS
    SELECT {col_clause}
      FROM gdp_raw
     WHERE "Country Name" IN ({countries_sql});
""")
cur.execute("DROP TABLE gdp_raw;")
cur.execute("ALTER TABLE gdp_filtered RENAME TO gdp_raw;")
conn.commit()

# 4) Filter emp_raw → emp_filtered, drop old, rename
cur.execute(f"""
    CREATE TABLE emp_filtered AS
    SELECT {col_clause}
      FROM emp_raw
     WHERE "Country Name" IN ({countries_sql});
""")
cur.execute("DROP TABLE emp_raw;")
cur.execute("ALTER TABLE emp_filtered RENAME TO emp_raw;")
conn.commit()

# 5) Filter gdp_growth_raw → gdp_growth_filtered, drop old, rename
cur.execute(f"""
    CREATE TABLE gdp_growth_filtered AS
    SELECT {col_clause}
      FROM gdp_growth_raw
     WHERE "Country Name" IN ({countries_sql});
""")
cur.execute("DROP TABLE gdp_growth_raw;")
cur.execute("ALTER TABLE gdp_growth_filtered RENAME TO gdp_growth_raw;")
conn.commit()

# 6) Verify first 5 rows of each
for tbl in ['gdp_raw','emp_raw','gdp_growth_raw']:
    print(f"\n--- {tbl} ---")
    cur.execute(f"SELECT {col_clause} FROM {tbl} LIMIT 5;")
    rows = cur.fetchall()
    # print column headers then rows
    print(columns)
    for row in rows:
        print(row)

# 7) Close connection
conn.close()



--- gdp_raw ---
['Country Name', 'Indicator Name', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']
('Canada', 'GDP per capita (constant 2015 US$)', 31700.5462697892, 30654.3331377446, 30563.3788184233, 31032.7546013139, 32074.1562430473, 32595.4515181692, 32801.3018549155, 33867.0079131743, 34894.6134280252, 36391.5353258957, 37906.8601153293, 38200.4561131358, 38921.6672955178, 39270.0234544073, 40108.758738975, 41006.222931999, 41663.5123054377, 42106.8724267286, 42067.5686998623, 40376.4153677037, 41164.3399062659, 42043.6419607016, 42320.6431282192, 42851.1705330235, 43643.2356474848, 43594.1941045394, 43551.3426018259, 44339.3886692751, 44907.3436837248, 45100.2914903262, 42366.1285192527, 44359.6167266713, 45227.1447354204, 44468.7475023539)
('China', 'GDP pe

We assemble three indicators from the World Bank’s World Development Indicators:

GDP per capita (constant 2015 US$)

Employment to population ratio

Annual GDP growth (%)

For each, we focus on six high- and middle-income economies (China, Japan, South Korea, the United States, United Kingdom, Canada) over the period 1990–2023. We will merge these into a single country–year panel, clean and format, and then explore summary statistics for each indicator.